# Chapter 7: Compose

## MuseGAN

_MuseGAN_ is a generative machine learning model capable of generating new samples of _multiphonic_ music, i.e. it is capable of composing music with multiple tracks. This is in contrast to the LSTM with an attension mechanism in `AttentionMechanism.ipynb`, which can only generate _monophonic_ music.

MuseGAN, which was introduced in [this paper](https://arxiv.org/abs/1709.06298), like other GANs, consists of a pair of convolutional neural networks, a generator and a critic. The MuseGAN we are building will be generating two new bars of choral music, using music composed by Bach for training.

## Data Preparation

### Downloading the Data

Here I download the training data from a [this GitHub repository](https://github.com/czhuang/JSB-Chorales-dataset). For this repository I will use a fork of the repository at the time that I am writing this notebook.

In [1]:
!git clone https://github.com/DCtheTall/JSB-Chorales-dataset

Cloning into 'JSB-Chorales-dataset'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36
Unpacking objects: 100% (36/36), done.


### Preprocessing the Data

The function below outputs the raw data, the data as a 4D tensor in the shape `[n_songs, n_bars, n_steps_per_bar, n_tracks]`. We then one hot encode the data into a 5D tensor of the shape `[n_songs, n_bars, n_steps_per_bar, n_notes, n_tracks]`.

In [0]:
import numpy as np

def load_music(filename, n_bars, n_steps_per_bar):
  """Load the training data into memory and preprocess it."""
  with np.load(filename, encoding='bytes', allow_pickle=True) as f:
    data = f['train']

  data_ints = []
  timesteps = n_bars * n_steps_per_bar
  
  for x in data:
    counter = 0
    while np.any(np.isnan(x[counter:(counter + 4)])):
      counter += 4
    if timesteps < x.shape[0]:
      data_ints.append(x[counter:(counter + timesteps), :])

  data_ints = np.array(data_ints)
  n_songs, _, n_tracks = data_ints.shape
  
  data_ints = data_ints.reshape((n_songs, n_bars, n_steps_per_bar, n_tracks))

  max_note = 83
  where_nans = np.isnan(data_ints)
  data_ints[where_nans] = max_note + 1
  max_note += 1

  data_ints = data_ints.astype(np.int)
  n_classes = max_note + 1

  data_binary = np.eye(n_classes)[data_ints]  # One-hot encode the pitches
  data_binary[data_binary == 0] = -1  # Replace 0s with -1s.
  # Remove the index indicating the last possible pitch. For that pitch, we
  # can just use a row of all -1's.
  data_binary = np.delete(data_binary, max_note, -1)
  data_binary = data_binary.transpose((0, 1, 2, 4, 3))

  return np.squeeze(data_binary), data_ints, data

In [0]:
N_BARS = 2
N_STEPS_PER_BAR = 16

data_binary, data_ints, data = load_music(
    'JSB-Chorales-dataset/Jsb16thSeparated.npz', N_BARS, N_STEPS_PER_BAR)